# Totito (Tic-Tac-Toe) con Minimax Search

In [1]:
from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import display
from functools import partial
import numpy as np

## Funciones de Ayuda

In [2]:
def get_next_turn(active_turn):
    # Change player
    # 1 -> o
    #-1 -> x
    if active_turn == 1:
        next_turn = -1
    else:
        next_turn = 1
    return next_turn

def score(matrix, i_am, depth):
    # Returns +-10 points -+ depth
    # It is the score for user i_am depending on matrix
    # If matrix correspond to a win for user i_am, get 10 points - depth
    # If matrix correspond to a win for the other user, return depth - 10
    # If matrix if not a winning condition, then returns 0
    # The maximum depth is 8 so it seems natural that the max points are 10
    points = 10
    status = game_status(matrix)
    if status==0:
        return 0
    if status==i_am:
        return points - depth
    if status!=i_am:
        return depth - points
    
def get_childs(matrix, turn):
    # turn is 1 or -1
    # 1 -> o
    #-1 -> x
    # 0 -> Free space
    # return all posible plays for user 'turn' ('x' or 'o')
    N = 3
    childs = []
    for i in range(N):
        for j in range(N):
            if matrix[i,j]==0:
                child = matrix.copy()
                child[i,j] = turn
                childs.append(child) 
    return childs

def game_status(matrix):
    # Returns 1 if 'o' win, -1 if 'x' win, 0 if draw 
    points = 1
    if (matrix[0,:].sum() == 3)|(matrix[1,:].sum() == 3)|(matrix[2,:].sum() == 3)|(matrix[:,0].sum() == 3)|(matrix[:,1].sum() == 3)|(matrix[:,2].sum() == 3):
        return points
    if (matrix[0,0]==matrix[1,1])&(matrix[2,2]==matrix[1,1])&(matrix[0,0]==1):
        return points
    if (matrix[0,2]==matrix[1,1])&(matrix[2,0]==matrix[1,1])&(matrix[2,0]==1):
        return points
    if (matrix[0,:].sum() == -3)|(matrix[1,:].sum() == -3)|(matrix[2,:].sum() == -3)|(matrix[:,0].sum() == -3)|(matrix[:,1].sum() == -3)|(matrix[:,2].sum() == -3):
        return -points
    if (matrix[0,0]==matrix[1,1])&(matrix[2,2]==matrix[1,1])&(matrix[0,0]==-1):
        return -points
    if (matrix[0,2]==matrix[1,1])&(matrix[2,0]==matrix[1,1])&(matrix[2,0]==-1):
        return -points
    return 0

def game_over(matrix):
    # status <- Returns 1 if 'o' win, -1 if 'x' win, 0 if draw 
    # game_finished: true is game is over
    game_finished = False
    status = game_status(matrix)
    if status!=0:
        #Game finishes, someone won
        game_finished = True
    if abs(matrix).sum() == 9:
        #No more moves
        game_finished = True
    return game_finished, status

def maximize(matrix, active_turn, player, depth, alpha, beta, nodes_visited):
    game_finished,_ = game_over(matrix)
    if game_finished:
        return None, score(matrix, player, depth), nodes_visited
    depth += 1
    
    infinite_number = 100000
    maxUtility = -infinite_number
    choice = None
    
    childs = get_childs(matrix, active_turn)
    for child in childs:
        nodes_visited = nodes_visited + 1
        _, utility, nodes_visited = minimize(child, get_next_turn(active_turn), player, depth, alpha, beta, nodes_visited)
        
        if utility > maxUtility:
            choice = child
            maxUtility = utility
                
        if maxUtility >= beta:
            break
        if maxUtility > alpha:
            alpha = maxUtility
    return choice, maxUtility, nodes_visited

def minimize(matrix, active_turn, player, depth, alpha, beta, nodes_visited):
    game_finished,_ = game_over(matrix)
    if game_finished:
        return None, score(matrix, player, depth), nodes_visited
    depth += 1
    infinite_number = 100000
    minUtility = infinite_number
    choice = None
    
    childs = get_childs(matrix, active_turn)
    for child in childs:
        nodes_visited = nodes_visited + 1
        _, utility, nodes_visited = maximize(child, get_next_turn(active_turn), player, depth, alpha, beta, nodes_visited)
        
        if utility < minUtility:
            choice = child
            minUtility = utility
            
        if minUtility <= alpha:
            break
        if minUtility < beta:
            beta = minUtility
    return choice, minUtility, nodes_visited

def minimax(matrix, player):
    infinite_number = 1000
    alpha = -infinite_number
    beta = infinite_number
    choice, score, nodes_visited = maximize(matrix, player, player, 0, alpha, beta, 0)
    return choice, score, nodes_visited

## Interfaz del Juego

References:
http://neverstopbuilding.com/minimax

In [3]:
class General_functions(object):
    def __init__(self, matrix, actual_turn):
        self.N = 3
        self.button_list = None
        self.text_box = None
        self.matrix = matrix
        self.game_finished = False
        self.actual_turn = actual_turn
        
    def display_matrix(self):
        N = self.N
        childs = []
        for i in range(N):
            for j in range(N):
                if self.matrix[i,j]==1:
                    self.button_list[i*N + j].description = 'o'
                if self.matrix[i,j]==-1:
                    self.button_list[i*N + j].description = 'x'

    def on_button_clicked(self, index, button):
        N = self.N  

        if self.game_finished:
            return

        y = index%N
        x = int(index/N)
        if self.matrix[x,y]!=0:
            self.text_box.value = 'No se puede ahi!'
            return
        button.description = self.actual_turn[0]

        if self.actual_turn == 'o':
            self.matrix[x,y] = 1
            self.game_finished, status = game_over(self.matrix)
            if self.game_finished:
                if (status!=0):
                    self.text_box.value = 'o wins'
                else: 
                    self.text_box.value = 'draw'
            else:
                self.actual_turn = 'x'
                self.text_box.value = 'Juega '+self.actual_turn
        else:
            self.matrix[x,y] = -1
            self.game_finished, status = game_over(self.matrix)
            if self.game_finished:
                if (status!=0):
                    self.text_box.value = 'x wins'
                else: 
                    self.text_box.value = 'draw'
            else:
                self.actual_turn = 'o'
                self.text_box.value = 'Juega '+self.actual_turn
        self.computer_play()
        
    def draw_board(self):
        self.text_box = widgets.Text(value = 'Juega '+self.actual_turn, layout=Layout(width='129px', height='40px'))
        self.button_list = []
        for i in range(9):
            button = widgets.Button(description='',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            icon='',
            layout=Layout(width='40px', height='40px'))
            self.button_list.append(button)
            button.on_click(partial(self.on_button_clicked, i))
        tic_tac_toe_board = VBox([HBox([self.button_list[0],self.button_list[1],self.button_list[2]]),
                HBox([self.button_list[3],self.button_list[4],self.button_list[5]]),
                HBox([self.button_list[6],self.button_list[7],self.button_list[8]])])
        display(VBox([self.text_box, tic_tac_toe_board]))
        return

    def computer_play(self):

        if self.game_finished:
            return
        
        if self.actual_turn=='x':
            turn = -1
            next_turn = 'o'
        if self.actual_turn=='o':
            turn = 1
            next_turn = 'x'
        self.matrix = self.get_best_play(turn)
        self.display_matrix()
        self.actual_turn = next_turn
        self.text_box.value = 'Juega '+self.actual_turn
        self.game_finished, status = game_over(self.matrix)
        if self.game_finished:
            if (status!=0):
                self.text_box.value = 'computer wins'
            else: 
                self.text_box.value = 'draw'

    def get_best_play(self, turn):
        # 1000 is an infinite value compared with the highest cost of 10 that we can get

        choice, points, nodes_visited = minimax(self.matrix, turn)
        print('points:',points)
        print('nodes_visited:',nodes_visited)
        return choice

## Empieza un Juego nuevo

In [4]:
def start_game(computer_starts = True, user_icon='x', start_mode = 'center'):
    matrix = np.zeros((3,3))
    
    if user_icon=='x':
        computer_icon_representation = 1
    else:
        computer_icon_representation = -1

    GF = General_functions(matrix, user_icon)
    GF.draw_board()

    if computer_starts:
        if start_mode == 'center':
            matrix[1,1] = computer_icon_representation
        elif start_mode == 'minimax':
            GF.computer_play()
        elif start_mode == 'random':
            x = np.random.randint(3)
            y = np.random.randint(3)
            matrix[x,y] = computer_icon_representation

    GF.display_matrix()

In [5]:
# start_mode:
# 'minimax': Uses minimax to select the first move
# 'center': Starts on the center
# 'random': Starts on a random position
# user_icon:
#  'x': user is x
#  'o': user is o
# computer_starts: True or False

start_game(computer_starts = True, user_icon = 'x', start_mode = 'center')

points: 0
nodes_visited: 831
points: 0
nodes_visited: 104
points: 0
nodes_visited: 11
points: 0
nodes_visited: 1
